In [26]:
from ieeg.auth import Session
from IPython.display import Image

import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np

from scipy import signal as sig
import math

In [27]:
colors = ["#cec5c1", "#9f8f7f", "#924a42", "#5a2028",
        "#a4b9b1", "#4ea7a6", "#026d73", "#06494f"];

plt.rcParams['savefig.dpi'] = 300;

rc('font', **{'family':'serif', 'serif':['Computer Modern']})
rc('text', usetex = True)

In [46]:
with open('jrp_ieeglogin.bin', 'r') as f:
  session = Session('jrp24', f.read()); 

dataset = session.open_dataset('I521_Sub1_Training_ecog'); 

print(dataset.get_channel_indices); 

<bound method Dataset.get_channel_indices of Dataset with: 62 channels.>


In [29]:
def load_full_channels(dataset, duration_secs, sampling_rate, chn_idx):
  """
  Loads the entire channel from IEEG.org
  Input:
    dataset: the IEEG dataset object
    duration_secs: the duration of the channel, in seconds
    sampling_rate: the sampling rate of the channel, in Hz
    chn_idx: the indicies of the m channels you want to load, as an array-like object
  Returns:
    [n, m] ndarry of the channels' values.
  """
  #stores the segments of the channel's data
  chn_segments = []

  #how many segments do we expect?
  num_segments = int(np.ceil(duration_secs * sampling_rate / 6e5))

  #segment start times and the step
  seg_start, step = np.linspace(1, duration_secs*1e6, num_segments, endpoint=False, retstep=True)

  #get the segments
  for start in seg_start:
    chn_segments.append(dataset.get_data(start, step, chn_idx))

  #concatenate the segments vertically
  return np.vstack(chn_segments)


In [50]:
duration_us = dataset.end_time - dataset.start_time; # in microseconds
duration_s = duration_us * 1e-6; # in seconds
duration_min = duration_s / 60; # in seconds
duration_hr = duration_min / 60; # in seconds
duration_ms = duration_us * 1e-3; # in seconds

period = 0.001; 
frequency = 1 / period 

print(f"sec: {round(duration_s, 2)}, min: {round(duration_min, 2)}, hr: {round(duration_hr, 2)}, ms: {round(duration_ms, 2)}")

sec: 300.0, min: 5.0, hr: 0.08, ms: 299999.0


In [59]:
num_channels = 62; 
fig, axs = plt.subplots(num_channels, 1); 

fig.set_figwidth(15); 
fig.set_figheight(62); 

for i in range(62):
    signal = load_full_channels(dataset, duration_s, frequency, [i]);
    x_axis = np.linspace(0, len(signal), len(signal)) / frequency;

    axs[i].plot(x_axis, signal, color = colors[5], linewidth = 0.5);

    # axs[i].set_xlabel("Time (s)");
    # axs[i].set_ylabel("Voltage ($\mu$V)");

plt.tight_layout(); 
plt.subplots_adjust(left = None, bottom = None, right = None, top = None, wspace = None, hspace = None); 
plt.show(); 